# We explore this notebook and see how we can make modification to complete our tasks

Some ideas


*   Cross-gender identification
* Age identification
*   Deployment to mobile devices

Dataset provided in assignment brief: https://susanqq.github.io/UTKFace/

Download this: https://drive.google.com/drive/folders/0BxYys69jI14kU0I1YUQyY1ZDRUE?resourcekey=0-01Pth1hq20K4kuGVkp3oBw



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Age and Gender Estimation in TensorFlow ( Workbook 2, Gender Classification )**

* For Age Estimation, go to [Workbook 1 ( Age Estimation )](https://colab.research.google.com/drive/1to2iolQGIVZgXFWRmRKZGrny5kfXk40h?usp=sharing)

In this notebook, we train a Keras model to classify the gender of a person, given a *face-cropped* image. We use the famous [UTKFace Dataset](https://susanqq.github.io/UTKFace/), which contains 23K images where each image is labelled with its gender, age and ethinicity.


> **Note: Please make sure that you are connected to the GPU runtime of Google Colab. Else, the training might take a decade long. Go to Runtime > Change runtime type > Hardware accelerator.**

## 1) **Import libraries**

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import datetime
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
from tensorflow.keras.layers import *
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

## 2) **Processing the data**

Once we've downloaded the dataset, we need to perform the following operations on the dataset, so that it can be used for training our model,

* Reading the image files as 3D NumPy arrays. Note, we'll use 3-channeled RGB images for training the model, so each array will have a shape of `[ img_width , img_height , 3 ]`.

* Split the filename so as to parse the gender of the person in corresponding image. We use the `tf.strings.split()` method for performing this task.

* We one-hot encode the gender, as we'll perform *a two-class* classification.

Once this operations have been performed, we are left with $N$ samples where each sample consists of image array `[ 128 , 128 , 3 ]` and its corresponding label ( one-hot encoded ), the gender of that person, which has a shape `[ 1 , 2 ]`

We'll use `tf.data.Dataset` as it helps us to process the data faster, taking advantage of parallel computing. The above two operations will be mapped on each filename using `tf.data.Dataset.map` method.

In [2]:
# definitions for UTKDataset
dataset_dict = {
    'race_id': {
        0: 'white', 
        1: 'black', 
        2: 'asian', 
        3: 'indian', 
        4: 'others'
    },
    'gender_id': {
        0: 'male',
        1: 'female'
    }
}

dataset_dict['gender_alias'] = dict((g, i) for i, g in dataset_dict['gender_id'].items())
dataset_dict['race_alias'] = dict((r, i) for i, r in dataset_dict['race_id'].items())

In [18]:
dir = '/content/drive/MyDrive/Dataset/UTK/UTKFace'

# Image size for our model.
# MODEL_INPUT_IMAGE_SIZE = [ 128 , 128 ]

# TRAIN_TEST_SPLIT = 0.3

# # Number of samples to take from dataset
# NUM_SAMPLES = 20000

# # Trick to one-hot encode the label.
# y1 = tf.constant( [ 1. , 0. ] , dtype='float32' ) 
# y2 = tf.constant( [ 0. , 1. ] , dtype='float32' ) 

# # This method will be mapped for each filename in `list_ds`. 
# def parse_image(filename):
#     # Read the image from the filename and resize it.
#     image_raw = tf.io.read_file(filename)
#     image = tf.image.decode_jpeg(image_raw ,channels=3) 
#     image = tf.image.resize(image ,MODEL_INPUT_IMAGE_SIZE) / 255

#     # Split the filename to get the age and the gender. Convert the age ( str ) and the gender ( str ) to dtype float32.
#     parts = tf.strings.split( tf.strings.split( filename , '/' )[ 2 ] , '_' )

#     # One-hot encode the label
#     gender = tf.strings.to_number(parts[1])
#     gender = ( gender * y2 ) + ( ( 1 - gender ) * y1 )
#     print(gender)

#     return image, 2

# # List all the image files in the given directory.
# list_ds = tf.data.Dataset.list_files( f'{dir}/*' , shuffle=True )
# # Map `parse_image` method to all filenames.
# dataset = list_ds.map(parse_image, num_parallel_calls=tf.data.AUTOTUNE, deterministic=False)
# # dataset = list_ds.map(lambda x: tf.py_function(func=parse_image, inp=[x], Tout=(tf.float32, tf.float32)),num_parallel_calls=tf.data.AUTOTUNE, deterministic=False)
# dataset = dataset.take( NUM_SAMPLES )
# print(dataset)

# for e in dataset:
#     print(e)

# Fraction of the dataset to be used for testing.
split = 0.3

batch_size = 128

image_data = []

def parse_data(filename):
  try:
    parts = filename.split('_')
    age = parts[0]
    gender = parts[1]
    race = parts[2]
    return int(age), dataset_dict['gender_id'][int(gender)], dataset_dict['race_id'][int(race)], filename
  except Exception as e:
    return None, None, None, None

for i in os.listdir(dir):
  age, gender, race, filename = parse_data(i)
  if age is not None and gender is not None and race is not None and filename is not None:
    image_data.append(parse_data(i))

data = pd.DataFrame(image_data, columns=["age", "gender", "race", "filename"])

# 1-hot embedding
# data['Age'] = tf.keras.utils.to_categorical(data['Age'])
# data['Gender'] = tf.keras.utils.to_categorical(data['Gender'])
# data['Racevalues'] = tf.keras.utils.to_categorical(data['Racevalues'])

# train test split
train, test = train_test_split(data, test_size = split)
train.head(10)


,age,gender,race,filename
20529,21,female,asian,21_1_2_20170104021056028.jpg.chip.jpg
7211,35,male,white,35_0_0_20170117171138059.jpg.chip.jpg
19601,26,female,indian,26_1_3_20170104215700398.jpg.chip.jpg
14753,56,female,asian,56_1_2_20170107213838566.jpg.chip.jpg
20085,23,male,white,23_0_0_20170104004006925.jpg.chip.jpg
20481,24,female,asian,24_1_2_20170104020244348.jpg.chip.jpg
10652,28,female,asian,28_1_2_20170116161415319.jpg.chip.jpg
10687,28,female,asian,28_1_2_20170116164552683.jpg.chip.jpg
3010,36,male,white,36_0_0_20170117184847906.jpg.chip.jpg
16275,15,female,white,15_1_0_20170109214633067.jpg.chip.jpg


In [9]:
def plot_distribution(pd_series):
    labels = pd_series.value_counts().index.tolist()
    counts = pd_series.value_counts().values.tolist()
    
    pie_plot = go.Pie(labels=labels, values=counts, hole=.3)
    fig = go.Figure(data=[pie_plot])
    fig.update_layout(title_text='Distribution for %s' % pd_series.name)
    
    fig.show()

In [ ]:
plot_distribution(train['gender'])
plot_distribution(train['race'])

In [ ]:
fig = px.histogram(train, x="age", nbins=20)
fig.update_layout(title_text='Age distribution on Train Dataset')
fig.show()

In [ ]:
plot_distribution(test['gender'])
plot_distribution(test['race'])

In [ ]:
fig = px.histogram(test, x="age", nbins=20)
fig.update_layout(title_text='Age distribution on Test Dataset')
fig.show()

Face dataset generator

In [19]:
img_height = 128
img_width = 128

class FaceDataGenerator():
  def __init__(self, dataframe):
    self.dataframe = dataframe

  def preprocess_image(self, file):
    img = Image.open(file)
    img = img.resize((img_width, img_height))
    img = np.array(img) / 255.0
    return img
  
  def alias(self):
    self.dataframe['gender_id'] = self.dataframe['gender'].map(lambda gender: dataset_dict['gender_alias'][gender.strip()])
    self.dataframe['race_id'] = self.dataframe['race'].map(lambda race: dataset_dict['race_alias'][race])
    self.max_age = self.dataframe['age'].max()

  def generate_images(self, is_training, batch_size=8):
    # arrays to store our batched data
    images, ages, races, genders = [], [], [], []
    while True:
      for idx in range(len(self.dataframe)):
          person = self.dataframe.iloc[idx]
          
          age = person['age']
          race = person['race_id']
          gender = person['gender_id']
          filename = person['filename']
          
          im = self.preprocess_image(dir + '/' + filename)
          
          ages.append(age / self.max_age)
          races.append(to_categorical(race, len(dataset_dict['race_id'])))
          genders.append(to_categorical(gender, len(dataset_dict['gender_id'])))
          images.append(im)
          
          # yielding condition
          if len(images) >= batch_size:
              yield np.array(images), [np.array(ages), np.array(races), np.array(genders)]
              images, ages, races, genders = [], [], [], []
              
      if not is_training:
          break

In [20]:
train_gen = FaceDataGenerator(train)
test_gen = FaceDataGenerator(test)
train_gen.alias()
test_gen.alias()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [101]:

# testdatagenerator = ImageDataGenerator(rescale=1. /255)
# testdata = testdatagenerator.flow_from_dataframe(dataframe=test,directory=dir,x_col="Filepath",y_col=["Age","Gender","Racevalues"],class_mode="multi_output", batch_size=batch_size)

# traindatagenerator = ImageDataGenerator(rescale=1. /255,shear_range =0.2,zoom_range=0.2,horizontal_flip =True)
# traindata = traindatagenerator.flow_from_dataframe(dataframe=train,directory=dir,x_col="Filepath",y_col=["Age","Gender","Racevalues"],class_mode="multi_output", batch_size=batch_size)


In [ ]:
# # Create train and test splits of the dataset.
# num_examples_in_test_ds = int( dataset.cardinality().numpy() * TRAIN_TEST_SPLIT )

# test_ds = dataset.take( num_examples_in_test_ds )
# train_ds = dataset.skip( num_examples_in_test_ds )

# print( 'Num examples in train ds {}'.format( train_ds.cardinality() ) )
# print( 'Num examples in test ds {}'.format( test_ds.cardinality() ) )


Num examples in train ds 14000
Num examples in test ds 6000



## 3) Model

Our aim is to develop a model which has lesser parameters ( which implies lesser inference time and size ) but powerful enough so that it can generalize better.

* The model takes in a batch of shape `[ None , 128 , 128 , 3 ]` and performs a number of convolutions on it as determined by `num_blocks`.
* Each block consists of a sequence of layers : `Conv2D -> BatchNorm -> LeakyReLU`



* If `lite_model` is set to `True`, we use [Separable Convolutions](https://towardsdatascience.com/a-basic-introduction-to-separable-convolutions-b99ec3102728) which have lesser parameters. We could achieve a *faster* model, compromising its performance.

* We stack such`num_blocks` blocks sequentially, where the no. of filters for each layer is taken from `num_filters`.

* Next we add a number of `Dense` layers to learn the features extracted by convolutional layers. Note, we also add a `Dropout` layer, to reduce overfitting. The `rate` for each `Dropout` layer is decreased subsequently for each layer, so that the learnability of `Dense` layer with lesser units ( neurons ) is not affected.

* The last `Dense` layer applies the softmax activation function which yields a probability distribution for the two classes `male` and `female`.

> See [this](https://machinelearningmastery.com/how-to-reduce-overfitting-in-deep-learning-with-weight-regularization/) blog for choosing the weight decay values used in the above two blocks.

* 👉🏻 The output of the model is a tensor with shape `[ None, 2 ]`


In [ ]:
# Custom preprocessing layer to modify image brightness randomly.
class RandomBrightness( tf.keras.layers.Layer ):

    def __init__( self , max_delta ):
        super( RandomBrightness , self ).__init__()
        self.__max_delta = max_delta

    def call( self , inputs ):
        return tf.image.random_brightness( inputs , self.__max_delta )

    def get_config( self ):
        return { "max_delta" : self.__max_delta }


In [17]:
MODEL_INPUT_IMAGE_SIZE = [128, 128]
# Negative slope coefficient for LeakyReLU.
leaky_relu_alpha = 0.2

lite_model = True

# Define the conv block.
def conv( x , num_filters , kernel_size=( 3 , 3 ) , strides=1 ):
    if lite_model:
        x = tf.keras.layers.SeparableConv2D( num_filters ,
                                            kernel_size=kernel_size ,
                                            strides=strides, 
                                            use_bias=False ,
                                            kernel_initializer=tf.keras.initializers.HeNormal() ,
                                            kernel_regularizer=tf.keras.regularizers.L2( 1e-5 )
                                             )( x )
    else:
        x = tf.keras.layers.Conv2D( num_filters ,
                                   kernel_size=kernel_size ,
                                   strides=strides ,
                                   use_bias=False ,
                                   kernel_initializer=tf.keras.initializers.HeNormal() ,
                                   kernel_regularizer=tf.keras.regularizers.L2( 1e-5 )
                                    )( x )

    x = tf.keras.layers.BatchNormalization()( x )
    x = tf.keras.layers.LeakyReLU( leaky_relu_alpha )( x )
    return x

def dense( x , filters , dropout_rate ):
    x = tf.keras.layers.Dense( filters , kernel_regularizer=tf.keras.regularizers.L2( 0.1 ) , bias_regularizer=tf.keras.regularizers.L2( 0.1 ) )( x )
    x = tf.keras.layers.LeakyReLU( alpha=leaky_relu_alpha )( x )
    x = tf.keras.layers.Dropout( dropout_rate )( x )
    return x

# No. of convolution layers to be added.
num_blocks = 5

# Num filters for each conv layer.
num_filters = [ 16 , 32 , 64 , 128 , 256 , 256 ]

# Kernel sizes for each conv layer.
kernel_sizes = [ 3 , 3 , 3 , 3 , 3 , 3 ]

# Init a Input Layer.
inputs = tf.keras.layers.Input(shape=MODEL_INPUT_IMAGE_SIZE + [3])

def standard_conv_layers(inputs):
  x = Conv2D(16, (3, 3), padding="same")(inputs)
  x = Activation("relu")(x)
  x = BatchNormalization(axis=-1)(x)
  x = MaxPooling2D(pool_size=(3, 3))(x)
  x = Dropout(0.25)(x)

  x = Conv2D(32, (3, 3), padding="same")(x)
  x = Activation("relu")(x)
  x = BatchNormalization(axis=-1)(x)
  x = MaxPooling2D(pool_size=(2, 2))(x)
  x = Dropout(0.25)(x)

  x = Conv2D(32, (3, 3), padding="same")(x)
  x = Activation("relu")(x)
  x = BatchNormalization(axis=-1)(x)
  x = MaxPooling2D(pool_size=(2, 2))(x)
  x = Dropout(0.25)(x)

  return x
#################### Gender Classifier ###########################
# for i in range( num_blocks ):
#     x = conv( x , num_filters=num_filters[ i ] , kernel_size=kernel_sizes[ i ] )
#     x = tf.keras.layers.MaxPooling2D()( x )

# Flatten the output of the last Conv layer.
x = inputs
x = Lambda(lambda c: tf.image.rgb_to_grayscale(c))(inputs)
x = standard_conv_layers(x)
x = Flatten()(x)
x = Dense(128)(x)
x = Activation("relu")(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
gender = Dense(2, activation='sigmoid', name='gender_output')(x)
# x = tf.keras.layers.Flatten()( x )
# conv_output = x 
# Add Dense layers ( Dense -> LeakyReLU -> Dropout )
# x = dense(conv_output , 256 , 0.6 )
# x = dense(x ,64 ,0.4 )
# x = dense(x ,32 ,0.2 )
# gender = tf.keras.layers.Dense(2, activation='sigmoid', name='gender')(x)
#################### End of Gender Classifier ###########################

#################### Race Classifier ###########################
# for i in range( num_blocks ):
#     x = conv( x , num_filters=num_filters[ i ] , kernel_size=kernel_sizes[ i ] )
#     x = tf.keras.layers.MaxPooling2D()( x )

# Flatten the output of the last Conv layer.
# x = tf.keras.layers.Flatten()( x )
# conv_output = x 

# # Add Dense layers ( Dense -> LeakyReLU -> Dropout )
# x = dense(conv_output , 256 , 0.6 )
# x = dense(x ,64 ,0.4 )
# x = dense(x ,32 ,0.2 )
x = inputs
x = standard_conv_layers(inputs)
x = Flatten()(x)
x = Dense(128)(x)
x = Activation("relu")(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
race = tf.keras.layers.Dense(5, activation='softmax', name='race_output')(x)
#################### End of Race Classifier ###########################

#################### Age Classifier ###########################
# for i in range( num_blocks ):
#     x = conv( x , num_filters=num_filters[ i ] , kernel_size=kernel_sizes[ i ] )
#     x = tf.keras.layers.MaxPooling2D()( x )

# # Flatten the output of the last Conv layer.
# x = tf.keras.layers.Flatten()( x )
# conv_output = x 

# # Add Dense layers ( Dense -> LeakyReLU -> Dropout )
# x = dense(conv_output , 256 , 0.6 )
# x = dense(x ,64 ,0.4 )
# x = dense(x ,32 ,0.2 )
x = inputs
x = standard_conv_layers(inputs)
x = Flatten()(x)
x = Dense(128)(x)
x = Activation("relu")(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
age = tf.keras.layers.Dense(1, activation='linear', name='age_output')(x)
#################### End of Age Classifier ###########################

# Build the Model
model = tf.keras.models.Model(inputs , [age, race, gender])

# Uncomment the below to view the summary of the model.
# model.summary()
# tf.keras.utils.plot_model(model) # add to_file='architecture.png' to export architecture 



Run this cell to visualize the training of the model in TensorBoard ( in this notebook itself ).

In [ ]:
%load_ext tensorboard
%tensorboard --logdir tb_logs/

## 4) **Compiling the model ( and other callbacks )** 🧱

Once we've defined the architecture for our model, we'll compile our Keras model and also initialize some useful callbacks.

* As we're performing classification, we'll use the Categorical Crossentropy loss function. See [`tf.keras.losses.CategoricalCrossentropy`](https://www.tensorflow.org/api_docs/python/tf/keras/losses/CategoricalCrossentropy) for more details.

* We'll use the Adam optimizer for training our model. See [`tf.keras.optimizers.Adam`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam) for more details.

* For evaluating the performance of our model, we measure the accuracy of our model. See [`tf.keras.metrics.Accuracy`](https://www.tensorflow.org/api_docs/python/tf/keras/metrics/Accuracy) for more details.


#### Callbacks:

* [`tf.keras.callbacks.ModelCheckpoint`](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint) to save the Keras model as an H5 file after every epoch.

* [`tf.keras.callbacks.TensorBoard`](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/TensorBoard) to visualize the training with TensorBoard.

* [`tf.keras.callbacks.EarlyStopping`](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping) to stop the training when the evaluation metric i.e the MAE stops improving on the test dataset.

In [21]:
learning_rate = 1e-4
num_epochs = 10 
# batch_size = 2

# train_ds = train_ds.batch(batch_size).repeat(num_epochs)
# test_ds = test_ds.batch(batch_size).repeat(num_epochs)

save_dir = 'checkpoints/cp.ckpt'
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint( save_dir )
logdir = os.path.join( "logs/tb_logs" , datetime.datetime.now().strftime("%Y%m%d-%H%M%S") )
tensorboard_callback = tf.keras.callbacks.TensorBoard( logdir )
early_stopping_callback = tf.keras.callbacks.EarlyStopping( monitor='val_accuracy' , patience=3 )

model.compile(
    optimizer = tf.keras.optimizers.Adam(
        learning_rate=learning_rate,
        decay=learning_rate/num_epochs
    ),
    loss = {
        'age_output': 'mse',
        'race_output': 'categorical_crossentropy',
        'gender_output': 'binary_crossentropy'
    },
    metrics = {
        'age_output': 'mae',
        'race_output': 'accuracy',
        'gender_output': 'accuracy'
    }
)


## 5) **Train and Evaluate the Model** 🏋🏻‍♂️

Start the training loop with all callbacks packed in.


In [ ]:
batch_size = 32

train_data = train_gen.generate_images(is_training = True, batch_size = batch_size)
test_data = test_gen.generate_images(is_training = True, batch_size = batch_size)

model.fit( 
    train_data,
    epochs=num_epochs,  
    validation_data=test_data,
    callbacks=[checkpoint_callback , tensorboard_callback , early_stopping_callback]
)


Epoch 1/10
    179/Unknown - 2574s 14s/step - loss: 6.9344 - age_output_loss: 3.8591 - race_output_loss: 2.2421 - gender_output_loss: 0.8333 - age_output_mae: 1.5492 - race_output_accuracy: 0.3202 - gender_output_accuracy: 0.6358

Evaluate the Model.

In [ ]:

p = model.evaluate( test_ds )
print( 'loss is {} \n accuracy is {} %'.format( p[0] , p[1] * 100 ) )



Save the Keras model to the local disk, so that we can resume training if needed.



In [ ]:

model_name = 'model_gender_augmented' #@param {type: "string"}
model_name_ = model_name + '.h5'

model.save( model_name_ )
files.download( model_name_ ) 



## 6) **Visualize the results**

We'll predict the age from some images taken from `test_ds` and plot them using `matplotlib`.


In [ ]:

fig = plt.figure( figsize=( 12 , 15 ) )
classes = [ 'Male' , 'Female' ]
rows = 5
columns = 2

i = 1
for image , label in test_ds.unbatch().take( 10 ):
    image = image.numpy()
    fig.add_subplot( rows , columns , i )
    plt.imshow( image )
    label_ = classes[ np.argmax( model.predict( np.expand_dims( image , 0 ) ) ) ]
    plt.axis( 'off' )
    plt.title( 'Predicted gender : {} , actual gender : {}'.format( label_ , classes[ np.argmax( label ) ] ) )
    i += 1


 
## 7) **Convert to TensorFlow Lite format** 📡

Our model is to be deployed in an Android app, where we'll use [TF Lite Android](https://bintray.com/google/tensorflow/tensorflow-lite) package to parse the model and make predictions.

We use the `TFLiteConverter` API to convert our Keras Model ( `.h5` ) to a TF Lite buffer ( `.tflite` ). See the [official docs](https://www.tensorflow.org/api_docs/python/tf/lite/TFLiteConverter/). We'll produce two TF Lite buffers, one with float16 quantization and other non-quantized model.


In [ ]:

converter = tf.lite.TFLiteConverter.from_keras_model( model )
converter.optimizations = [ tf.lite.Optimize.DEFAULT ]
converter.target_spec.supported_types = [ tf.float16 ]
buffer = converter.convert()

open( '{}_q.tflite'.format( model_name ) , 'wb' ).write( buffer )
files.download( '{}_q.tflite'.format( model_name ) )



For conversion to a non-quantized TF Lite buffer.


In [ ]:

converter = tf.lite.TFLiteConverter.from_keras_model( model )
buffer = converter.convert()

open( '{}_nonq.tflite'.format( model_name ) , 'wb' ).write( buffer )
files.download( '{}_nonq.tflite'.format( model_name ) )



## Utility Methods

Use these methods to automate some of the tasks.


In [ ]:

#@title Utility to zip and download a directory
#@markdown Use this method to zip and download a directory. For ex. a TB logs 
#@markdown directory or a checkpoint(s) directory.

dir_to_zip = 'tb_logs' #@param {type: "string"}
output_filename = 'logs.zip' #@param {type: "string"}
delete_dir_after_download = "No"  #@param ['Yes', 'No']

os.system( "zip -r {} {}".format( output_filename , dir_to_zip ) )

if delete_dir_after_download == "Yes":
    os.system( "rm -r {}".format( dir_to_zip ) )

files.download( output_filename )


In [ ]:

#@title Utility to delete a directory
#@markdown Use this method to delete a directory. 

dir_path = ''  #@param {type: "string"}
os.system( f'rm -r {dir_path}')
